# From the solar wind to the ground

> Abstract: We demonstrate a basic analysis of a geomagnetic storm using hapiclient & viresclient to access data from the solar wind (OMNI IMF), Low Earth Orbit (Swarm-derived auroral electrojet estimates), and the ground (INTERMAGNET observatory magnetic measurements).


## Packages to use

- [`hapiclient`](https://github.com/hapi-server/client-python) to access solar wind data from [OMNI](https://omniweb.gsfc.nasa.gov/) (alternatively we could use [`pysat`](https://pysat.readthedocs.io/en/latest/quickstart.html))
    - For more examples with hapiclient, take a look at [the demonstration notebooks](https://github.com/hapi-server/client-python-notebooks)
- [`viresclient`](https://github.com/ESA-VirES/VirES-Python-Client/) to access AEJs from Swarm, and B-field from ground observatories
- [`xarray`](https://xarray.pydata.org/) and [`matplotlib`](https://matplotlib.org/) for data wrangling and plotting
    - See the [xarray tutorial website](https://xarray-contrib.github.io/xarray-tutorial/) to learn more

In [ ]:
%load_ext watermark
%watermark -i -v -p viresclient,hapiclient,pandas,xarray,matplotlib

In [ ]:
from copy import deepcopy
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

from viresclient import SwarmRequest
from hapiclient import hapi, hapitime2datetime

## Time selection

Let's choose an interesting time period to study - the ["St. Patrick's day storm" of 17th March 2015](https://doi.org/10.1186/s40623-016-0525-y). You can look at the wider context of this event using the interactive [Space Weather Data Portal from the University of Colorado](https://lasp.colorado.edu/space-weather-portal/data/display?active-range=%5B1425967200000,1426831200000%5D&outer-range=%5B1262552105447,1559362748308%5D&plots=%5B%7B%22datasets%22:%7B%22sdo_eve_diodes_l2%22:%5B%22diode171%22%5D%7D,%22options%22:%7B%7D%7D,%7B%22datasets%22:%7B%22sdo_aia_0094_0335_0193_image_files%22:%5B%22url%22%5D%7D,%22options%22:%7B%7D%7D,%7B%22datasets%22:%7B%22ac_h0_mfi%22:%5B%22Magnitude%22%5D%7D,%22options%22:%7B%7D%7D,%7B%22datasets%22:%7B%22ac_h1_epm%22:%5B%22P7%22,%22P8%22%5D%7D,%22options%22:%7B%7D%7D,%7B%22datasets%22:%7B%22ac_h0_swe%22:%5B%22Vp%22%5D%7D,%22options%22:%7B%7D%7D,%7B%22datasets%22:%7B%22gracea_density%22:%5B%22neutral_density%22%5D%7D,%22options%22:%7B%7D%7D,%7B%22datasets%22:%7B%22usgs_geomag_brw_definitive%22:%5B%22X%22%5D,%22usgs_geomag_frn_definitive%22:%5B%22X%22%5D,%22usgs_geomag_cmo_definitive%22:%5B%22X%22%5D%7D,%22options%22:%7B%7D%7D,%7B%22datasets%22:%7B%22usgs_geomag_brw_definitive%22:%5B%22Y%22%5D,%22usgs_geomag_frn_definitive%22:%5B%22Y%22%5D,%22usgs_geomag_cmo_definitive%22:%5B%22Y%22%5D%7D,%22options%22:%7B%7D%7D,%7B%22datasets%22:%7B%22usgs_geomag_brw_definitive%22:%5B%22Z%22%5D,%22usgs_geomag_frn_definitive%22:%5B%22Z%22%5D,%22usgs_geomag_cmo_definitive%22:%5B%22Z%22%5D%7D,%22options%22:%7B%7D%7D,%7B%22datasets%22:%7B%22swt_bfield_maps%22:%5B%22url%22%5D%7D,%22options%22:%7B%7D%7D,%7B%22datasets%22:%7B%22swt_efield_maps%22:%5B%22url%22%5D%7D,%22options%22:%7B%7D%7D,%7B%22datasets%22:%7B%22swt_voltage_maps%22:%5B%22url%22%5D%7D,%22options%22:%7B%7D%7D%5D)

We will use the same time window to fetch data from the different sources:

In [ ]:
START_TIME = '2015-03-15T00:00:00Z'
END_TIME = '2015-03-20T00:00:00Z'

## Solar wind data (OMNI)

HAPI is an access protocol supported by a wide array of heliophysics datasets. We can use the Python package "hapiclient" to retrieve data from HAPI servers. In this case we will access the [OMNI HRO2 dataset](https://omniweb.gsfc.nasa.gov/html/HROdocum.html) which provides consolidated solar wind data, and then we will show how we can load these data into pandas and xarray objects.

> OMNI Combined, Definitive 1-minute IMF and Definitive Plasma Data Time-Shifted to the Nose of the Earth's Bow Shock, plus Magnetic Indices - J.H. King, N. Papatashvilli (AdnetSystems, NASA GSFC)

To generate code snippets to use, and to see what data are available:  
http://hapi-server.org/servers/#server=CDAWeb&dataset=OMNI_HRO2_1MIN&parameters=flow_speed&start=2000-01-01T00:00:00Z&stop=2000-02-01T00:00:00Z&return=script&format=python

Here we will access five-minute-resolution measurements of the Interplanetary Magnetic Field (IMF) vector and the bulk flow speed of the solar wind:

In [ ]:
def fetch_omni_data(start, stop):
    server     = 'https://cdaweb.gsfc.nasa.gov/hapi'
    dataset    = 'OMNI_HRO2_5MIN'
    parameters = 'BX_GSE,BY_GSM,BZ_GSM,flow_speed';
    data, meta = hapi(server, dataset, parameters, start, stop)
    return data, meta

data, meta = fetch_omni_data(START_TIME, END_TIME)

Data are automatically loaded as a [NumPy structured array](https://numpy.org/doc/stable/user/basics.rec.html) and metadata as a dictionary:

In [ ]:
data

In [ ]:
meta

We are now able to extract an array for a particular value like `data["BZ_GSM"]`, and use the metadata to get full descriptions and units for the chosen parameter.

The metadata sometimes contains fill values used during data gaps (e.g. the 9999... values appearing above). Let's use those to replace the gaps with NaN values:

In [ ]:
def fill2nan(hapidata_in, hapimeta):
    """Replace bad values (fill values given in metadata) with NaN"""
    hapidata = deepcopy(hapidata_in)
    # HAPI returns metadata for parameters as a list of dictionaries
    # - Loop through them
    for metavar in hapimeta['parameters']:  
        varname = metavar['name']
        fillvalstr = metavar['fill']
        if fillvalstr is None:
            continue
        vardata = hapidata[varname]
        mask = vardata==float(fillvalstr)
        nbad = np.count_nonzero(mask)
        print('{}: {} fills NaNd'.format(varname, nbad))
        vardata[mask] = np.nan
    return hapidata, hapimeta

data, meta = fill2nan(data,meta)
data

We can load the data into a pandas DataFrame to more readily use for analysis:

In [ ]:
def to_pandas(hapidata):
    df = pd.DataFrame(
        columns=hapidata.dtype.names,
        data=hapidata,
    ).set_index("Time")
    # Convert from hapitime to Python datetime
    df.index = hapitime2datetime(df.index.values.astype(str))
    # df.index = pd.DatetimeIndex(df.index.values.astype(str))
    # Remove timezone awareness
    df.index = df.index.tz_convert("UTC").tz_convert(None)
    # Rename to Timestamp to match viresclient
    df.index.name = "Timestamp"
    return df

df = to_pandas(data)
df

How can we get the extra information like the units from the metadata? Let's construct dictionaries, `units` and `description`, that allow easier access to these:

In [ ]:
def get_units_description(meta):
    units = {}
    description = {}
    for paramdict in meta["parameters"]:
        units[paramdict["name"]] = paramdict.get("units", None)
        description[paramdict["name"]] = paramdict.get("description", None)
    return units, description
units, description = get_units_description(meta)
units, description

The [`xarray.Dataset`](http://xarray.pydata.org/en/stable/data-structures.html#dataset) object has advantages for handling multi-dimensional data and for attaching of metadata like units. Let's convert the data to an `xarray.Dataset`:

In [ ]:
def to_xarray(hapidata, hapimeta):
    # Here we will conveniently re-use the pandas function we just built,
    # and use the pandas API to build the xarray Dataset.
    # NB: if performance is important, it's better to build the Dataset directly
    ds = to_pandas(hapidata).to_xarray()
    units, description = get_units_description(hapimeta)
    for param in ds:
        ds[param].attrs = {
            "units": units[param],
            "description": description[param]
        }
    return ds

ds_sw = to_xarray(data, meta)
ds_sw

Now let's plot these data:

In [ ]:
def plot_solar_wind(ds_sw):
    fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(15, 5))
    for IMF_var in ["BX_GSE", "BY_GSM", "BZ_GSM"]:
        ds_sw[IMF_var].plot.line(
            x="Timestamp", linewidth=1, alpha=0.8, ax=axes[0], label=IMF_var
        )
    axes[0].legend()
    axes[0].set_ylabel("IMF\n[nT]")
    axes[0].set_xlabel("")
    ds_sw["flow_speed"].plot.line(
        x="Timestamp", linewidth=1, alpha=0.8, ax=axes[1]
    )
    axes[1].set_ylabel("flow_speed\n[km/s]")
    axes[1].set_xlabel("")
    axes[0].grid()
    axes[1].grid()
    fig.suptitle("Interplanetary Magnetic Field and Solar Wind flow")
    return fig, axes

fig_sw, axes_sw = plot_solar_wind(ds_sw)

## Auroral electrojets as measured by Swarm

Since spacecraft move, it is difficult to extract a simple time series that can be easily tracked. From the complex Swarm product portfolio, we will pick a particular derived parameter: the peak auroral electrojet intensities derived from each pass over the current system. This signal tracks reasonably well from one orbit to the next (when separated into four orbital segments - accounting for two passes over the auroral oval in different local time sectors, and over the northern and southern hemispheres).

To keep things a bit simpler, we will retrieve data only from Swarm Alpha over the Northern Hemisphere. The auroral electrojet peaks and boundaries for Swarm Alpha are contained within the product named `SW_OPER_AEJAPBL_2F`. Here is how we can access these data:

In [ ]:
def fetch_Swarm_AEJ(start_time, end_time):
    request = SwarmRequest()
    # Meaning of AEJAPBL: (AEJ) Auroral electrojets
    #                     (A)   Swarm Alpha
    #                     (PBL) Peaks and boundaries from LC method
    # J_QD is the current intensity along QD-latitude contours
    # QDOrbitDirection is a flag (1, -1) marking the direction of the 
    #   satellite (ascending, descending) relative to the QD pole
    # MLT is magnetic local time, evaluated according to the
    #   quasi-dipole magnetic longitude and the sub-solar point
    #   (see doi.org/10.1007/s11214-016-0275-y)
    request.set_collection("SW_OPER_AEJAPBL_2F")
    request.set_products(
        measurements=["J_QD", "PointType"],
        auxiliaries=["QDOrbitDirection", "MLT"]
    )
    # PointType of 0 refers to WEJ (westward electrojet) peaks
    # PointType of 1 refers to EEJ (eastward electrojet) peaks
    # See https://nbviewer.jupyter.org/github/pacesm/jupyter_notebooks/blob/master/AEBS/AEBS_00_data_access.ipynb#AEJxPBL-product
    request.set_range_filter("Latitude", 0, 90)  # Northern hemisphere
    request.set_range_filter("PointType", 0, 1)  # Extract only peaks
    data = request.get_between(START_TIME, END_TIME, asynchronous=False, show_progress=False)
    ds_AEJ_peaks = data.as_xarray()
    return ds_AEJ_peaks

ds_AEJ_peaks = fetch_Swarm_AEJ(START_TIME, END_TIME)
ds_AEJ_peaks

:::{tip}
Switch `asynchronous=False` to `asynchronous=True` if making longer requests
:::

Now we need some complex logic to plot the eastward and westward electrojet intensities, separated for each local time sector:

In [ ]:
def plot_AEJ_envelope(ds_AEJ_peaks):
    # Masks to identify which sector the satellite is in
    #  and which current type (WEJ/EEJ) is given
    mask_asc = ds_AEJ_peaks["QDOrbitDirection"] == 1
    mask_desc = ds_AEJ_peaks["QDOrbitDirection"] == -1
    mask_WEJ = ds_AEJ_peaks["PointType"] == 0
    mask_EEJ = ds_AEJ_peaks["PointType"] == 1

    fig, axes = plt.subplots(nrows=2, sharex=True, sharey=True, figsize=(15, 5))
    # Select and plot from the ascending orbital segments
    #  on axes 0
    # Eastward electrojet:
    _ds = ds_AEJ_peaks.where(mask_EEJ & mask_asc, drop=True)
    _ds["J_QD"].plot.line(x="Timestamp", ax=axes[0], label="EEJ")
    # Westward electrojet:
    _ds = ds_AEJ_peaks.where(mask_WEJ & mask_asc, drop=True)
    _ds["J_QD"].plot.line(x="Timestamp", ax=axes[0], label="WEJ")
    # Identify approximate MLT of sector
    _ds = ds_AEJ_peaks.where(mask_asc, drop=True)
    mlt = round(float(_ds["MLT"].mean()))
    axes[0].set_ylabel(axes[0].get_ylabel() + f"\nMLT: ~{mlt}")
    # ... and for descending segments
    #  on axes 1
    # Eastward electrojet:
    _ds = ds_AEJ_peaks.where(mask_EEJ & mask_desc, drop=True)
    _ds["J_QD"].plot.line(x="Timestamp", ax=axes[1], label="EEJ")
    # Westward electrojet:
    _ds = ds_AEJ_peaks.where(mask_WEJ & mask_desc, drop=True)
    _ds["J_QD"].plot.line(x="Timestamp", ax=axes[1], label="WEJ")
    # Identify approximate MLT of sector
    _ds = ds_AEJ_peaks.where(mask_desc, drop=True)
    mlt = round(float(_ds["MLT"].mean()))
    axes[1].set_ylabel(axes[1].get_ylabel() + f"\nMLT: ~{mlt}")
    axes[1].legend()
    axes[0].set_xlabel("")
    axes[1].set_xlabel("")
    axes[0].grid()
    axes[1].grid()
    fig.suptitle("Auroral electrojet envelope measured by Swarm Alpha")
    return fig, axes

fig_aej, axes_aej = plot_AEJ_envelope(ds_AEJ_peaks)

This shows us the envelope of the auroral electrojet system - how the strength of the Eastward (EEJ) and Westward (WEJ) electrojets evolve over time - but only over the two local time sectors that the spacecraft is moving through. The strengths of the electric current along the contours of Quasi-Dipole latitude, `J_QD`, have been calculated.

### Peak ground magnetic disturbances below satellite tracks

Swarm also provides predictions of the location and strength of the peak disturbance on the ground (along the satellite ground-track) caused by the auroral electrojets. Note that this is from the AEJ_PBS (using the SECS method) collection rather than the AEJ_PBL (using the LC method) used above.

In [ ]:
def fetch_Swarm_AEJ_disturbances(start_time, end_time):
    request = SwarmRequest()
    request.set_collection("SW_OPER_AEJAPBS_2F:GroundMagneticDisturbance")
    request.set_products(
        measurements=["B_NE"],
        auxiliaries=["OrbitNumber", "QDOrbitDirection"]
    )
    request.set_range_filter("Latitude", 0, 90)  # Northern hemisphere only
    data = request.get_between(start_time, end_time, asynchronous=False, show_progress=False)
    ds = data.as_xarray()
    # Add vector magnitude
    ds["B_Total"] = "Timestamp", np.sqrt((ds["B_NE"].data**2).sum(axis=1))
    ds["B_Total"].attrs["units"] = "nT"
    return ds

ds_AEJ_disturbances = fetch_Swarm_AEJ_disturbances(START_TIME, END_TIME)
ds_AEJ_disturbances

This dataset contains two samples per pass over each half of the auroral oval, estimating the ground location of the peak magnetic disturbance due to each of the EEJ and WEJ currents, and the associated strength (`B_NE`) of the North and East components of the disturbance. Let's look at an approximation of the overall strongest ground disturbances, by inspecting the maximum strength found over 90-minute windows (i.e. approximately each orbit):

In [ ]:
def plot_Swarm_ground_disturbance(ds_AEJ_disturbances):
    fig, ax = plt.subplots(figsize=(15, 3))
    ds_resample = ds_AEJ_disturbances.resample({'Timestamp':'90Min'}).max()
    ds_resample["B_Total"].plot.line(x="Timestamp", ax=ax)
    fig.suptitle("Peak ground disturbance estimated from Swarm Alpha")
    ax.set_ylabel("Magnetic disturbance\n[nT]")
    ax.set_xlabel("")
    ax.grid()
    return fig, ax

fig_Sw_ground, ax_Sw_ground = plot_Swarm_ground_disturbance(ds_AEJ_disturbances)

## Ground observatory data (INTERMAGNET)

> We acknowledge usage of INTERMAGNET data  
> See <https://intermagnet.github.io/data_conditions.html> for more

As well as access to Swarm data, VirES also provides access to ground observatory data from INTERMAGNET. We can fetch data from the minute resolution dataset (`SW_OPER_AUX_OBSM2_`), specifying desired observatories according to their [3-letter IAGA codes](https://www.intermagnet.org/imos/imomap-eng.php). These data have been rotated from the geodetic reference frame to the geocentric frame (NEC).

We'll select three observatories in Sweden: Abisko (ABK), Lycksele (LYC) and Uppsala (UPS), which form a chain across about 10 degrees of latitude along a similar longitude.

In [ ]:
def fetch_ground_obs(IAGA_codes, start_time, end_time):
    request = SwarmRequest()
    request.set_collection(*[f"SW_OPER_AUX_OBSM2_:{c}" for c in IAGA_codes], verbose=False)
    request.set_products(
        measurements=["B_NEC", "IAGA_code"],
    )
    data = request.get_between(start_time, end_time, asynchronous=False, show_progress=False)
    ds = data.as_xarray(reshape=True)
    return ds

ds_ground_obs = fetch_ground_obs(["ABK", "LYC", "UPS"], START_TIME, END_TIME)
ds_ground_obs

By specifiying `reshape=True` when loading the xarray object, a multi-dimensional dataset is formed with a new `IAGA_code` axis. Here we show the three vector components from each observatory:

In [ ]:
ds_ground_obs["B_NEC"].plot.line(x="Timestamp", row="NEC", col="IAGA_code", sharey=False);

Let's calculate $|\frac{dB}{dt}|$ and plot that instead. This is a good indication of the GIC risk, as a more rapidly changing magnetic field will induce a larger electric field in the ground.

In [ ]:
def plot_groundobs_dbdt(ds_ground_obs):
    ds_ground_obs = ds_ground_obs.assign(
        dBdt=(ds_ground_obs["B_NEC"].diff("Timestamp")**2).sum(dim="NEC").pipe(np.sqrt)
    )
    fig, axes = plt.subplots(nrows=3, figsize=(15, 7), sharey=True, sharex=True)
    for i, obs in enumerate(ds_ground_obs["IAGA_code"].values):
        _ds = ds_ground_obs.sel(IAGA_code=obs)
        lat = np.round(float(_ds["Latitude"]), 1)
        lon = np.round(float(_ds["Longitude"]), 1)
        label = f"{obs} (Lat {lat}, Lon {lon})"
        ds_ground_obs["dBdt"].sel(IAGA_code=obs).plot.line(x="Timestamp", ax=axes[i], label=label)
        axes[i].set_title("")
        axes[i].legend()
        axes[i].set_xlabel("")
        axes[i].set_ylabel("dB/dt\n[nT / min]")
        axes[i].grid()
        fig.tight_layout()
    return fig, axes
    
fig_grdbdt, axes_grdbdt = plot_groundobs_dbdt(ds_ground_obs)